In [1]:
#imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
C_PATH = '../Data/country/'
countries = os.listdir(C_PATH)
#countries.remove('results')
print(countries)

COUNTRY_LIST = []
for c in countries:
    c_code = c[:2]
    COUNTRY_LIST.append(c_code)
print(COUNTRY_LIST)
print(len(COUNTRY_LIST))

['ie.csv', 'lk.csv', 'ng.csv', 'us.csv', 'au.csv', 'gh.csv', 'tz.csv', 'pk.csv', 'hk.csv', 'my.csv', 'jm.csv', 'ca.csv', 'ke.csv', 'bd.csv', 'sg.csv', 'in.csv', 'nz.csv', 'ph.csv', 'za.csv', 'gb.csv']
['ie', 'lk', 'ng', 'us', 'au', 'gh', 'tz', 'pk', 'hk', 'my', 'jm', 'ca', 'ke', 'bd', 'sg', 'in', 'nz', 'ph', 'za', 'gb']
20


In [3]:
TOPIC_LIST = ['ENVIRONMENT/ENERGY','INTERNATIONAL','POLITICS','EDUCATION/FAMILY', \
              'SPORTS', 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY', \
              'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY', \
              'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW','HEALTH/MEDICAL']
for ind, val in enumerate(TOPIC_LIST):
    print(ind, val)

0 ENVIRONMENT/ENERGY
1 INTERNATIONAL
2 POLITICS
3 EDUCATION/FAMILY
4 SPORTS
5 TECHNOLOGY/SCIENCE/SOCIAL MEDIA
6 SOCIAL_LIFE/DAILY
7 ENTERTAINMENT/ART/MAGAZINE
8 COMPANY/BUSINESS
9 ECONOMY
10 POLICE/ACCIDENT/VIOLENCE
11 LEGAL/LAW
12 HEALTH/MEDICAL


In [4]:
COUNTYR_DICT = {} ##### Assigned topics: {topic_id_from_list : percentage} #####

COUNTYR_DICT['tz'] = {0:{6:90,12:10}, 1:{0:100}, 2:{1:50,2:50}, 3:{4:100}, 4:{5:100}, 5:{11:100}, 6:{8:50,12:50}} 
COUNTYR_DICT['bd'] = {0:{10:100}, 1:{1:50, 2:50}, 2:{1:100}, 3:{0:100}, 4:{6:100}, 5:{5:100}, 6:{11:100}}
COUNTYR_DICT['gh'] = {0:{0:50,12:50}, 1:{1:40,4:60}, 2:{8:50, 9:50}, 3:{4:30,7:70}, 4:{11:100}, 5:{2:100}, 6:{7:100}} 
COUNTYR_DICT['hk'] = {0:{5:30,8:70}, 1:{2:100}, 2:{6:100}, 3:{1:100}, 4:{8:50,9:50}, 5:{10:100}, 6:{7:100}}

COUNTYR_DICT['sg'] = {0:{7:100}, 1:{6:100}, 2:{10:100}, 3:{5:20,6:60,9:20}, 4:{8:100}, 5:{0:20,6:50,12:30}, 6:{5:100}} 
COUNTYR_DICT['pk'] = {0:{6:100}, 1:{2:100}, 2:{1:50,2:50}, 3:{1:100}, 4:{9:100}, 5:{2:50,1:50}, 6:{4:50,5:20,7:30}} 
COUNTYR_DICT['lk'] = {0:{0:100}, 1:{8:100}, 2:{2:40,10:60}, 3:{1:100}, 4:{5:100}, 5:{4:100}, 6:{6:100}} 
COUNTYR_DICT['my'] = {0:{5:100}, 1:{11:100}, 2:{2:90,10:10}, 3:{9:100}, 4:{0:100}, 5:{6:100}, 6:{5:60,8:40}} 

COUNTYR_DICT['nz'] = {0:{7:100}, 1:{8:10,11:90}, 2:{0:100}, 3:{4:100}, 4:{10:20,11:80}, 5:{12:100}, 6:{6:100}} 
COUNTYR_DICT['au'] = {0:{7:100}, 1:{4:100}, 2:{1:20,11:80}, 3:{12:100}, 4:{6:100}, 5:{9:100}, 6:{5:100}} 
COUNTYR_DICT['ca'] = {0:{12:100}, 1:{6:50,7:50}, 2:{10:100}, 3:{6:50,8:50}, 4:{0:100}, 5:{9:100}, 6:{0:30,4:70}} 
COUNTYR_DICT['gb'] = {0:{1:20,12:80}, 1:{4:50,6:50}, 2:{1:100}, 3:{7:10,11:90}, 4:{4:100}, 5:{7:100}, 6:{8:80,9:20}} 

COUNTYR_DICT['ng'] = {0:{4:100}, 1:{0:100}, 2:{1:50,2:50}, 3:{10:80,11:20}, 4:{7:100}, 5:{6:70,7:30}, 6:{9:100}} 
COUNTYR_DICT['za'] = {0:{1:50,2:50}, 1:{0:10,6:90}, 2:{1:50,12:50}, 3:{2:10,11:90}, 4:{5:50,12:50}, 5:{4:100}, 6:{9:100}} 

COUNTYR_DICT['ie'] = {0:{4:100}, 1:{11:100}, 2:{9:100}, 3:{6:50,12:50}, 4:{2:40,4:60}, 5:{2:100}, 6:{7:100}}
COUNTYR_DICT['in'] = {0:{2:100}, 1:{8:80,11:20}, 2:{9:100}, 3:{7:100}, 4:{1:20,4:80}, 5:{5:100}, 6:{2:100}}
COUNTYR_DICT['jm'] = {0:{0:20,12:80}, 1:{2:30,10:70}, 2:{8:60,12:40}, 3:{10:50,11:50}, 4:{6:100}, 5:{4:100}, 6:{7:100}} 
COUNTYR_DICT['ke'] = {0:{8:100}, 1:{8:50,9:50}, 2:{1:50,4:50}, 3:{6:100}, 4:{4:10,5:70,6:10,10:10}, 5:{6:20, 7:80}, 6:{10:50,11:50}}

COUNTYR_DICT['ph'] = {0:{2:100}, 1:{12:50}, 2:{0:100}, 3:{10:100}, 4:{9:100}, 5:{4:100}, 6:{6:100}}
COUNTYR_DICT['us'] = {0:{2:100}, 1:{2:40,10:60}, 2:{0:20,12:80}, 3:{6:70,11:30}, 4:{4:100}, 5:{8:50,9:50}, 6:{5:60,7:40}}

In [5]:
#print(source_data[source_data.textID==1958181])

## Read source data and asignment data

In [6]:
# Read news source folder as a whole
source_data = pd.read_table('../Data/now_sources_full.txt',encoding = "ISO-8859-1", header=None)
source_data.rename(columns={0:'textID', 1:'#words',2:'date',3:'country',4:'website',5:'url',6:'title'},inplace=True)
print(len(source_data))
source_data.dropna(inplace=True)
source_data.date =  '20'+source_data.date
source_data.date =  pd.to_datetime(source_data.date, format='%Y-%m-%d')

6132175


In [7]:
len(source_data)

6132163

In [8]:
# Assign topics to specific docs accord
def assign_percentage_topics(topic_frame, topic_dist):
    #print(topic_dist)
    #print(topic_frame)
    slices = []
    row_start = 0
    row_end = 0
    percen = 0
    for t_id, t_dis in topic_dist.items():
        
        percen = (percen+t_dis)/100
        row_end = row_start+  int(len(topic_frame)*percen)
        mini_slice = topic_frame[row_start:row_end]
        mini_slice.Topics = TOPIC_LIST[t_id]
        
        row_start = row_end
        
        #print(mini_slice)
        slices.append(mini_slice)
    
    return pd.concat(slices) 


In [9]:
def get_article_by_country(c_code, c_dict):
    assignment_data = pd.read_csv(C_PATH+ c_code +'.csv', header=None)
    assignment_data.rename(columns={0:'textID', 1:'Topics'},inplace=True)
    topic_frames = []
    xx = []
    for assign_topic in c_dict.keys():
        t_fr = assign_percentage_topics(assignment_data[assignment_data.Topics==assign_topic],\
                                        c_dict[assign_topic])
        topic_frames.append(t_fr)
        #assigment_data.replace({"Topics": c_dict}, inplace=True)
    return pd.concat(topic_frames),xx

In [10]:
c_frames = []
for cind in range(len(COUNTRY_LIST)) :
    c_code = COUNTRY_LIST[cind]
    c_dict = COUNTYR_DICT[c_code]
    per_country,xx = get_article_by_country(c_code, c_dict)
    c_frames.append(per_country)

all_counries = pd.concat(c_frames)

In [11]:
#tz_data = pd.read_csv(C_PATH+ 'tz' +'.csv', header=None)
#tz_data.rename(columns={0:'textID', 1:'Topics'},inplace=True)
# assignment_data[assignmnent_data.Topics==2].Topics.value_counts()

In [12]:
all_counries

,textID,Topics
3,11015886,SPORTS
8,13509295,SPORTS
30,13802579,SPORTS
46,13965351,SPORTS
47,13989969,SPORTS
53,14123903,SPORTS
67,14386093,SPORTS
80,14517135,SPORTS
86,14591121,SPORTS
90,14622011,SPORTS


In [13]:
full_data = pd.merge(source_data, all_counries, left_on='textID', right_on='textID')

In [14]:
full_data

,textID,#words,date,country,website,url,title,Topics
0,3732490,815,2015-11-01,US,NPR,http://www.npr.org/2015/11/01/450889721/the-ma...,The Madonna Of 115th Street Gets A Long-Awaite...,SOCIAL_LIFE/DAILY
1,3732492,258,2015-11-01,US,Huffington Post,http://www.huffingtonpost.com/entry/university...,University Of Louisville Sorry A Bunch Of Its ...,SOCIAL_LIFE/DAILY
2,3732496,489,2015-11-01,US,Bleacher Report,http://bleacherreport.com/articles/2584708-kar...,'Kareem: Minority of One' HBO Documentary Prev...,POLITICS
3,3732501,840,2015-11-01,US,VentureBeat,http://venturebeat.com/2015/11/01/what-big-ind...,What big industry will do to the Internet of T...,HEALTH/MEDICAL
4,3732502,470,2015-11-01,US,Tech Insider,http://www.techinsider.io/daenerys-game-of-thr...,Daenerys has been traveling in the wrong direc...,SPORTS
5,3732503,650,2015-11-01,US,Entertainment Weekly,http://www.ew.com/recap/the-real-housewives-of...,'Reunion Part 3',POLITICS
6,3732536,143,2015-11-01,US,New York Post,http://nypost.com/2015/11/01/pharmacy-accident...,Pharmacy accidentally gave kids bipolar pills ...,SOCIAL_LIFE/DAILY
7,3732542,305,2015-11-01,US,Huffington Post,http://www.huffingtonpost.com/entry/bill-clint...,Bill Clinton Calls For Peace On 20th Anniversa...,HEALTH/MEDICAL
8,3732543,405,2015-11-01,US,PC Gamer,http://www.pcgamer.com/the-binding-of-isaac-af...,The Binding of Isaac: Afterbirth is out now,TECHNOLOGY/SCIENCE/SOCIAL MEDIA
9,3732545,1041,2015-11-01,US,Boing Boing,http://boingboing.net/2015/11/01/the-incredibl...,The incredible treasures of the Octavia Butler...,TECHNOLOGY/SCIENCE/SOCIAL MEDIA


In [15]:
g_country_topic = full_data.groupby(by=['country','Topics'])['textID'].count()

g_country = g_country_topic.groupby(by=['country']).sum()

In [16]:
g_country_topic

country  Topics                         
AU       ECONOMY                             7630
         ENTERTAINMENT/ART/MAGAZINE          7405
         HEALTH/MEDICAL                      6868
         INTERNATIONAL                       1424
         LEGAL/LAW                           5694
         SOCIAL_LIFE/DAILY                   5434
         SPORTS                              6012
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA     7911
BD       ENVIRONMENT/ENERGY                   708
         INTERNATIONAL                        793
         LEGAL/LAW                            956
         POLICE/ACCIDENT/VIOLENCE             793
         POLITICS                             157
         SOCIAL_LIFE/DAILY                    601
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA      501
CA       COMPANY/BUSINESS                    4193
         ECONOMY                             7938
         ENTERTAINMENT/ART/MAGAZINE          4193
         ENVIRONMENT/ENERGY                 11193
         

In [17]:
g_percentage = g_country_topic/g_country

In [18]:
g_percentage

country  Topics                         
AU       ECONOMY                            0.157716
         ENTERTAINMENT/ART/MAGAZINE         0.153065
         HEALTH/MEDICAL                     0.141965
         INTERNATIONAL                      0.029435
         LEGAL/LAW                          0.117698
         SOCIAL_LIFE/DAILY                  0.112324
         SPORTS                             0.124271
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA    0.163525
BD       ENVIRONMENT/ENERGY                 0.157019
         INTERNATIONAL                      0.175870
         LEGAL/LAW                          0.212020
         POLICE/ACCIDENT/VIOLENCE           0.175870
         POLITICS                           0.034819
         SOCIAL_LIFE/DAILY                  0.133289
         TECHNOLOGY/SCIENCE/SOCIAL MEDIA    0.111111
CA       COMPANY/BUSINESS                   0.073039
         ECONOMY                            0.138273
         ENTERTAINMENT/ART/MAGAZINE         0.073039
     

In [19]:
topics_distribution = pd.DataFrame(full_data['country'].unique()) 
topics_distribution.rename(columns={0:'country'},inplace=True)
topics_distribution['country_name'] = ['United States', 
                                       'Canada', 
                                       'United Kingdom',
                                       'Ireland',
                                        'Australia',
                                        'New Zealand',
                                        'India',
                                        'Sri Lanka',
                                       'Pakistan', 
                                       'Bangladesh',
                                        'Malaysia',
                                        'Singapore',
                                        'Philippines',
                                       'Hong Kong',
                                       'South Africa',
                                        'Nigeria', 
                                        'Ghana',
                                        'Kenya',                           
                                        'Tanzania',
                                       'Jamaica']

In [20]:
for col in TOPIC_LIST:
    topics_distribution[col] = None

In [21]:
g_percentage_pd = pd.DataFrame(g_percentage)
g_percentage_pd.reset_index(inplace=True)
g_percentage_pd.head()

,country,Topics,textID
0,AU,ECONOMY,0.157716
1,AU,ENTERTAINMENT/ART/MAGAZINE,0.153065
2,AU,HEALTH/MEDICAL,0.141965
3,AU,INTERNATIONAL,0.029435
4,AU,LEGAL/LAW,0.117698


In [22]:
for ind, row in g_percentage_pd.iterrows():
    country = row.country
    topic = row.Topics
    freq = row.textID
    topics_distribution.loc[topics_distribution['country'] == country, topic] = freq

In [23]:
topics_distribution

,country,country_name,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,US,United States,0.022056,None,0.194978,None,0.113442,0.0828643,0.13251,0.0552429,0.0848248,0.084809,0.0845718,0.0566184,0.0880818
1,CA,Canada,0.194973,None,None,None,0.0958403,None,0.1466,0.0730386,0.0730386,0.138273,0.165221,None,0.113016
2,GB,United Kingdom,None,0.175466,None,None,0.22891,None,0.0572487,0.147708,0.148592,0.0372031,None,0.121004,0.0838684
3,IE,Ireland,None,None,0.177962,None,0.252041,None,0.0542922,0.184377,None,0.143317,None,0.13386,0.0541523
4,AU,Australia,None,0.0294349,None,None,0.124271,0.163525,0.112324,0.153065,None,0.157716,None,0.117698,0.141965
5,NZ,New Zealand,0.141424,None,None,None,0.185989,None,0.102319,0.133849,0.0174212,None,0.0308244,0.280272,0.107901
6,IN,India,None,0.0178418,0.347152,None,0.0707962,0.119554,None,0.132678,0.125539,0.155064,None,0.0313753,None
7,LK,Sri Lanka,0.234718,0.137635,0.0627247,None,0.122453,0.113064,0.115661,None,0.121055,None,0.0926888,None,None
8,PK,Pakistan,None,0.256281,0.321481,None,0.0711469,0.0288468,0.146365,0.0420775,None,0.133802,None,None,None
9,BD,Bangladesh,0.157019,0.17587,0.0348193,None,None,0.111111,0.133289,None,None,None,0.17587,0.21202,None


In [24]:
topics_distribution.to_csv('../Data/topic_distribution.csv', index=False)

In [25]:
full_data.to_csv('../Data/final-news-data.csv', index=False)